In [1]:
import scipy
import numpy as np
import matplotlib
import pandas as pd
import statsmodels
import sklearn
import tensorflow
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import string
import pyarrow as pa


Using TensorFlow backend.


Will put into csv first so that can randomly sort since combining so many songs from so many artists.

In [2]:
lyricsCSV = pd.read_csv('lyrics.csv',encoding='ISO-8859-1')
#lyricsSample = lyricsCSV.sample(frac=1)
#lyricsSample.head()
#lyricsSample.to_csv('lyrics.txt',sep='\t',index=False)
lyricsCSV.to_csv('lyrics.txt',sep='\t',index=False)

In [3]:
l = open('lyrics.txt','r')
lyrics = l.read()
l.close()

In [4]:
tokens = lyrics.split()
lyrics = ' '.join(tokens)
#lyrics = lyrics.translate(str.maketrans('','',string.punctuation)) #Remove punctuation because many songs from many sources

In [5]:
print(lyrics)

Sing a song of sixpence, A pocket full of rye. Four and twenty blackbirds, Baked in a pie. When the pie was opened The birds began to sing; Wasn't that a dainty dish, To set before the king. The king was in his counting house, Counting out his money; The queen was in the parlour, Eating bread and honey. The maid was in the garden, Hanging out the clothes, When down came a blackbird And pecked off her nose.


In [6]:
length = 10
sequences = list()
for i in range(length, len(lyrics)):
    seq = lyrics[i-length:i+1]
    sequences.append(seq)

In [7]:
data = '\n'.join(sequences)
file = open('char_sequences.txt','w')
file.write(data)
file.close()

In [8]:
file = open('char_sequences.txt','r')
raw_text = file.read()
file.close()

lines = raw_text.split('\n')

In [9]:
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))


In [10]:
sequences = list()
for line in lines:
    encoded_seq = [mapping[char] for char in line]
    sequences.append(encoded_seq)

In [11]:
vocab_size = len(mapping)
print('Vocabulary Size: {}'.format(vocab_size))

Vocabulary Size: 38


In [12]:

sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]


In [13]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [14]:
# define model   MAY NEED TO ADJUST 75
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                34200     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                2888      
Total params: 37,088
Trainable params: 37,088
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
#MAY NEED TO ADJUST 100

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 1s - loss: 3.6117 - acc: 0.1028
Epoch 2/100
 - 0s - loss: 3.4797 - acc: 0.1880
Epoch 3/100
 - 0s - loss: 3.1779 - acc: 0.1905
Epoch 4/100
 - 0s - loss: 3.0531 - acc: 0.1905
Epoch 5/100
 - 0s - loss: 3.0178 - acc: 0.1905
Epoch 6/100
 - 0s - loss: 2.9955 - acc: 0.1905
Epoch 7/100
 - 0s - loss: 2.9828 - acc: 0.1905
Epoch 8/100
 - 0s - loss: 2.9712 - acc: 0.1905
Epoch 9/100
 - 0s - loss: 2.9544 - acc: 0.1905
Epoch 10/100
 - 0s - loss: 2.9465 - acc: 0.1905
Epoch 11/100
 - 0s - loss: 2.9414 - acc: 0.1905
Epoch 12/100
 - 0s - loss: 2.9159 - acc: 0.1905
Epoch 13/100
 - 0s - loss: 2.8953 - acc: 0.1905
Epoch 14/100
 - 0s - loss: 2.8755 - acc: 0.1905
Epoch 15/100
 - 0s - loss: 2.8515 - acc: 0.1955
Epoch 16/100
 - 0s - loss: 2.8208 - acc: 0.2005
Epoch 17/100
 - 0s - loss: 2.7866 - acc: 0.2356
Epoch 18/100
 - 0s - loss: 2.7598 - acc: 0.2055
Epoch 19/100
 - 0s - loss: 2.7229 - acc: 0.2807
Epoch 20/100
 - 0s - loss: 2.6811 - acc: 0.2531
E

In [27]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_lyric, n_chars):
    lyrics = seed_lyric
    for __ in range(n_chars):
    # encode the characters as integers
        encoded = [mapping[char] for char in lyrics]
    # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
    # predict character
        yhat = model.predict_classes(encoded, verbose=0)
    # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
    # append to input
        lyrics += char
    return lyrics

In [28]:
print(generate_seq(model,mapping,10,'Song black',20))

Song blacktind Ah poeee. The i
